# PS 3 - Fall 2020 Lecture 7: Experiments


In this notebook, we will see how randomly picking treatment and control groups will minimize selection bias.

We will also get some practice with using data tables from the datascience library.

First, let's see see how our selection bias formula works with varying treatment effects.

We are going to import the now familiar numpy library, and also some function related to tables from the datascience library

In [2]:
import numpy as np
from datascience import Table

We are going to stick with the election monitoring example, but use some made up data to get practice with the key concepts. This first line of code defines the number of observations in our hypothetical data, which here correspond to elections. 

Let's assume the fraud levels without monitoring are uninformly distributed between 0 and 10; i.e., each number in this range is equally likely. Unlike last week, we are going to let the causal effect vary, in particular from -3 to 1 (and any number in that range is equally likely). 


In [3]:
# Setting the number of elections to 2000
n=2000
np.random.seed(32020)
# Creating a base "all seeing mode" table with potential outcomes w/o monitoring and treatment effect
fraud_asm = Table().with_columns(
    'y0', np.random.randint(0, 10, n),
    'k', np.random.randint(-3, 1, n)
)

In [4]:
# Peeking at the top of the table
fraud_asm

y0,k
5,-2
0,-3
9,-1
5,-1
5,0
6,0
2,-1
5,0
3,-3
6,-2


What is the average treatment effect in our data? To "pull" the causal effect from out data, we use square brackets and then include the variable name in quotation marks.

In [5]:
# We can compute the average treatment effect from our (hypothetical) data
np.mean(fraud_asm["k"])

-1.5015

Now let's compute the potential outcome with monitoring, which by definition is the potential outcome without monitoing plus the (individual-level) treatment effect. (Side note: this will sometimes give us numbers below 0 or above 10. To simplify I'll just leave this: think of them as EXTRA clean or EXTRA fraudlent elections).

In terms of the code, by saying "fraud_table["y1"]="... we are telling Python to add a new variable to fraud table called y1, and then set it equal to what we ask for on the right hand side of the equation. Under the hood, you can think of this is pulling out the y0 and k arrays, adding them to get a third array, and then adding that to our data table.

In [6]:
# Computing the potential outcome with monitoring
fraud_asm["y1"] = fraud_asm["y0"] + fraud_asm["k"]
fraud_asm

y0,k,y1
5,-2,3
0,-3,-3
9,-1,8
5,-1,4
5,0,5
6,0,6
2,-1,1
5,0,5
3,-3,0
6,-2,4


Now let's determine whether a country is monitored, and the realized outcome. To keep things conceptually clear, we'll do this in a new table for the "real" data.

To simplify, let's suppose monitors tend to go to countries where (1) the baseline level of fraud is high, and (2) they expect that their presence will reduce fraud. Specifically, they go to countries where the potential outcome with no monitoring is 5 or more, and the election-level treatment effect is negative.

In [28]:
fraud_real = Table().with_columns(
    'd', 1*(fraud_asm["y0"] >= 5)*(fraud_asm["k"] < 0))

# Once the monitoring presence is set, compute the realized outcome like before
fraud_real["y"] = fraud_real["d"]*fraud_asm["y1"] + (1 - fraud_real["d"])*fraud_asm["y0"]
fraud_real

d,y
1,3
0,0
1,8
1,4
0,5
0,6
0,2
0,5
0,3
1,4


Now let's look at the average of several things for the monitored elecitons...

In [29]:
np.mean(fraud_asm.where(fraud_real["d"]==1))

y0,k,y1
6.97055,-1.98079,4.98976


...vs non-monitored

In [30]:
np.mean(fraud_asm.where(fraud_real["d"]==0))

y0,k,y1
2.96719,-1.19442,1.77276


What do you notice in the differences?

Now let's check our formula works. Some of the code here is a bit ugly. First, we compute the difference of means, then the ATET and selection bias using the formulas from the slides.

In [31]:
avg_mon = np.mean(fraud_real.where(fraud_real["d"]==1)["y"])
avg_nm = np.mean(fraud_real.where(fraud_real["d"]==0)["y"])
dom = avg_mon - avg_nm
dom

2.022570503939439

In [25]:
atet = np.mean(fraud_asm.where(fraud_real["d"]==1)["y1"]) - np.mean(fraud_asm.where(fraud_real["d"]==1)["y0"])
atet

-1.9807938540332906

In [26]:
sb = np.mean(fraud_asm.where(fraud_real["d"]==1)["y0"]) - np.mean(fraud_asm.where(fraud_real["d"]==0)["y0"])
sb

4.00336435797273

Let's check that our difference of means is in fact the same as the ATET plus bias

In [27]:
dom, atet + sb

(2.022570503939439, 2.0225705039394395)

Notice our average treatment effect on everyone is a bit different:

In [15]:
ate = np.mean(fraud_asm["y1"]) - np.mean(fraud_asm["y0"])
ate

-1.5015

Now let's consider an alternate world where monitoring gets randomly assigned. We'll call this monitoring status d2, and the resulting outcome y2.

In [16]:
fraud_exp = Table().with_columns(
    'd', np.random.binomial(1, .5, n))
fraud_exp["y"] = fraud_exp["d"]*fraud_asm["y1"] + (1 - fraud_exp["d"])*fraud_asm["y0"]
fraud_exp

d,y
0,5
0,0
1,8
1,4
0,5
1,6
0,2
1,5
1,0
1,4


In [17]:
np.mean(fraud_asm.where(fraud_exp["d"]==1))

y0,k,y1
4.54275,-1.51297,3.02978


In [18]:
np.mean(fraud_asm.where(fraud_exp["d"]==0))

y0,k,y1
4.51721,-1.48905,3.02815


In [19]:
avg_mon_exp = np.mean(fraud_exp.where(fraud_exp["d"]==1)["y"])
avg_nm_exp = np.mean(fraud_exp.where(fraud_exp["d"]==0)["y"])
dom_exp = avg_mon_exp - avg_nm_exp
dom_exp

-1.4874263637174092

In [20]:
atet_exp = np.mean(fraud_asm.where(fraud_exp["d"]==1)["y1"]) - np.mean(fraud_asm.where(fraud_exp["d"]==1)["y0"])
atet_exp

-1.5129682997118152

Now our difference of means is very close to the ATET!

As an exercise, you may want to fiddle with the code and try decreasing/increasing the sample size.